In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [26]:
tf.__version__

'2.7.0'

### Load Dataset

In [29]:
dataset = pd.read_csv('/home/dewald/Downloads/MachineLearning/MachineLearning/datasets/Churn_Modelling.csv')


##check for missing data
dataset.isnull().sum(axis=0)

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [30]:

X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


### Encode catagorical data

In [32]:
## Label encoding

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

X[:,2] = encoder.fit_transform(X[:,2])

print(X)


[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [35]:
## one hot encoding

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder='passthrough')

X = np.array(ct.fit_transform(X))

print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Split data 

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2 , random_state = 0)

### Feature scaling 

In [50]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### only transform to test to avoid information leakage-
### never call fit on the test data

### Build ANN

In [51]:
ann = tf.keras.models.Sequential()

## adding layers

## First hidden layer
##....Dense({how many hidden layers})... relu= rectified linear activation function 
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## output layer
## binary output, units = 1 .. if catagorical (eg A, B or C) units = 3 
## activation func = sigmoid (give probalility)
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


### Compiling ANN

In [52]:
#optimizer : updates weights
#loss = for binary = binary_crossentropy, for nonbinary = catagorical_corssentropy

ann.compile(optimizer= 'adam' , loss= 'binary_crossentropy' , metrics= ['accuracy'])

### Training ANN

In [53]:
ann.fit(X_train,y_train,batch_size= 32, epochs = 100 )

### "An epoch means training the neural network with all the training data for one cycle"


Epoch 1/100
250/250 [==============================] - 0s 673us/step - loss: 0.5399 - accuracy: 0.7365
Epoch 2/100
250/250 [==============================] - 0s 728us/step - loss: 0.4582 - accuracy: 0.8020
Epoch 3/100
250/250 [==============================] - 0s 703us/step - loss: 0.4333 - accuracy: 0.8170
Epoch 4/100
250/250 [==============================] - 0s 678us/step - loss: 0.4179 - accuracy: 0.8255
Epoch 5/100
250/250 [==============================] - 0s 665us/step - loss: 0.4086 - accuracy: 0.8261
Epoch 6/100
250/250 [==============================] - 0s 752us/step - loss: 0.4016 - accuracy: 0.8291
Epoch 7/100
250/250 [==============================] - 0s 797us/step - loss: 0.3946 - accuracy: 0.8344
Epoch 8/100
250/250 [==============================] - 0s 860us/step - loss: 0.3874 - accuracy: 0.8384
Epoch 9/100
250/250 [==============================] - 0s 860us/step - loss: 0.3795 - accuracy: 0.8465
Epoch 10/100
250/250 [==============================] - 0s 721us/step - l

In [54]:
## input for predict must be a 2d array -- [[]]
## remember scaling
print(ann.predict(sc.transform([[1.0,0.0, 0.0, 600, 1, 40, 3,60000,2,1,1,50000]]) )> 0.5)

[[False]]


### Predict test set

In [55]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_pred),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Confusion matrix

In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)

accuracy_score(y_test,y_pred)

[[1501   94]
 [ 192  213]]


0.857